In [1]:
import os
import pandas as pd
from tqdm import tqdm

In [2]:
# Запустить, расскомментировать, запустить ещё раз (с speller и без)

In [3]:
titles = pd.read_csv('./titles_speller.tsv', sep='\t', header=None)
titles.columns=['Id', 'Title']

# titles = pd.read_csv('./titles_speller.tsv', sep='\t', header=None)
# titles.columns=['Id', 'Title']

In [4]:
titles

,Id,Title
0,334761,ответы mail ru кто родился 4 июля\n
1,56349,наращивание ногтей акрилом описание достоинств...
2,551990,как узнать есть ли что то в собственности zaza...
3,224905,relp криминалистическое исследование речевого ...
4,389288,николай козлов синтон психология тренинги розе...
...,...,...
582162,458053,голод сергей львович москвин голод книга подго...
582163,474067,как сделать фотографию снимок с экрана монитор...
582164,179069,селективная парфюмерия нишевая парфюмерия стат...
582165,539701,10 установленных плагинов wordpress в блоге но...


In [5]:
queries = pd.read_csv('./query_speller.tsv', sep='\t', header=None)
queries.columns = ['Id', 'Query']

# queries = pd.read_csv('./query_speller.tsv', sep='\t', header=None)
# queries.columns = ['Id', 'Query']

In [6]:
queries

,Id,Query
0,0,13 причин почему
1,1,1 положительный и 1 отрицательный могут ли
2,2,2016 действует ли зао рождественская мануфактура
3,3,1 месяц после операции на кишечнике диета что ...
4,4,2 правды 1 ложь что можно придумать
...,...,...
6306,6306,является ли тойота харриер внедорожником
6307,6307,як можно очистить крейду
6308,6308,являются ли реактивы медицинскими изделиями
6309,6309,являются ли словообразовательными парами слова...


In [7]:
queries['Id'] = queries['Id'].apply(lambda x: int(x))

In [8]:
titles['Title'] = titles['Title'].astype('str').apply(lambda x: x.strip())
titles['Id'] = titles['Id'].apply(lambda x: int(x))
titles = titles.sort_values(by=['Id'])

In [9]:
sample_df = pd.read_csv('./Data/sample.csv', sep=',')
sample_df.columns = ['query_id', 'url_id']

marks_df = pd.read_csv('./Data/train.marks.tsv', sep='\t', header=None)
marks_df.columns = ['query_id', 'url_id', 'mark']
marks_df = marks_df.drop(columns=['mark'])

sample_df = sample_df.append(marks_df)

sample_df['USE_qa'] = None
sample_df['USE_multi'] = None
sample_df['USE_large'] = None

In [10]:
corpus = titles['Title']

In [11]:
import tensorflow_hub as hub
import numpy as np
from tensorflow_text import SentencepieceTokenizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
model_qa = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3")
model_multi = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
model_large = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [12]:
all_q = list(set(sample_df['query_id']))

In [17]:
for query in tqdm(all_q):
    docs_id = list(sample_df[sample_df['query_id'] == query]['url_id'])
    
    q = queries[queries['Id'] == query]['Query'][query]

    q_vec_qa = model_qa([q])
    q_vec_multi = model_multi([q])
    q_vec_large = model_large([q])
    
    docs_qa = []
    docs_multi = []
    docs_large = []
        
    for batch in np.array_split(docs_id, 10):      
        docs_qa.extend(model_qa(corpus[batch]))
        docs_multi.extend(model_multi(corpus[batch]))
        docs_large.extend(model_large(corpus[batch]))

    use_qa = cosine_similarity(q_vec_qa, docs_qa)[0]
    use_multi = cosine_similarity(q_vec_multi, docs_multi)[0]
    use_large = cosine_similarity(q_vec_large, docs_large)[0]

    sample_df.loc[sample_df['query_id'] == query, 'USE_qa'] = use_qa
    sample_df.loc[sample_df['query_id'] == query, 'USE_multi'] = use_multi
    sample_df.loc[sample_df['query_id'] == query, 'USE_large'] = use_large

100%|██████████| 251/251 [02:17<00:00,  1.82it/s]


In [23]:
#sample_df.to_csv('USE/USE_Spelled.tsv',sep='\t',index=False)

In [18]:
sample_df.to_csv('USE/USE_Spelled_Stemmed.tsv',sep='\t',index=False)

### QA

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

In [10]:
sample_df = pd.read_csv('./Data/sample.csv', sep=',')
sample_df.columns = ['query_id', 'url_id']

marks_df = pd.read_csv('./Data/train.marks.tsv', sep='\t', header=None)
marks_df.columns = ['query_id', 'url_id', 'mark']
marks_df = marks_df.drop(columns=['mark'])

sample_df = sample_df.append(marks_df)

sample_df['USE_QA'] = None

In [13]:
corpus = titles['Title']

In [14]:
all_q = list(set(sample_df['query_id']))

In [15]:
for query in tqdm(all_q):
    docs_id = list(sample_df[sample_df['query_id'] == query]['url_id'])
    q = queries[queries['Id'] == query]['Query'][query]
    try:
        question_embeddings = module.signatures['question_encoder'](tf.constant(q))
        responses = corpus[docs_id]
        response_contexts = corpus[docs_id]
        response_embeddings = module.signatures['response_encoder'](
            input=tf.constant(responses),
            context=tf.constant(response_contexts))

        sim = np.inner(question_embeddings['outputs'], response_embeddings['outputs']).ravel()
        sample_df.loc[sample_df['query_id'] == query, 'USE_QA'] = sim
    except:
        question_embeddings = module.signatures['question_encoder'](tf.constant(q))
        sim = []
        for batch in np.array_split(docs_id, 100):      
            responses = corpus[batch]
            response_contexts = corpus[batch]
            response_embeddings = module.signatures['response_encoder'](
                input=tf.constant(responses),
                context=tf.constant(response_contexts))
            sim.extend(np.inner(question_embeddings['outputs'], response_embeddings['outputs']).ravel())
            
        sample_df.loc[sample_df['query_id'] == query, 'USE_QA'] = sim

100%|██████████| 6311/6311 [09:35<00:00, 10.97it/s]  


In [16]:
sample_df

,query_id,url_id,USE_QA
0,0,340485,0.038054
1,0,68106,0.038054
2,0,237314,0.063759
3,0,203791,0.02885
4,0,53265,0.028328
...,...,...,...
202074,6305,63981,0.01692
202075,6305,354802,0.029699
202076,6305,275960,0.031664
202077,6305,338427,0.040814


In [17]:
#sample_df.to_csv('USE/USEQA_Spelled.tsv',sep='\t',index=False)
sample_df.to_csv('USE/USEQA_Spelled_Stemmed.tsv',sep='\t',index=False)